In [ ]:
# ================================
# 1. Import bibliotek
# ================================
import os
import pandas as pd
from openai import OpenAI
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from catboost import CatBoostClassifier

# ================================
# 2. Konfiguracja API OpenAI
# ================================
# Opcja 1 (bezpieczniejsza): ustaw zmienną środowiskową OPENAI_API_KEY w systemie
# Opcja 2 (szybsza): wpisz klucz tutaj (NIE ZALECANE!)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ================================
# 3. Wczytanie danych
# ================================
df = pd.read_csv('/Users/annafurgala-wojas/New_project/Heart-disease-dataset/healthcare-dataset-stroke-data.csv')

# Rozdzielenie cech i targetu
X = df.drop("stroke", axis=1)
y = df["stroke"]

print("Rozkład klas przed resamplingiem:")
print(y.value_counts(normalize=True))

# ================================
# 4. Sugestie preprocessingowe z LLM
# ================================
prompt = f"""
Mam dataset z problemem niezbalansowanej klasy docelowej 'stroke':
{X.head().to_string()}

1. Jakie preprocessing steps mogą poprawić recall klasy mniejszościowej?
2. Jakie nowe cechy można stworzyć?
3. Które modele mogą działać najlepiej oprócz CatBoost?
Podaj gotowe przykłady kodu w Pythonie (Pandas).
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",  # lub gpt-5-mini jeśli masz dostęp
    messages=[
        {"role": "system", "content": "You are a senior data scientist."},
        {"role": "user", "content": prompt}
    ]
)

print("\n=== Sugestie od LLM ===\n")
print(response.choices[0].message.content)

# ================================
# 5. Oversampling ADASYN
# ================================
adasyn = ADASYN(random_state=42)
X_res, y_res = adasyn.fit_resample(X, y)

print("\nRozkład klas po ADASYN:")
print(y_res.value_counts(normalize=True))

# ================================
# 6. Train/test split
# ================================
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, random_state=42, stratify=y_res
)

# ================================
# 7. CatBoost – trening z optymalizacją recall
# ================================
model = CatBoostClassifier(
    iterations=1000,
    depth=6,
    learning_rate=0.05,
    loss_function='Logloss',
    eval_metric='Recall',   # kluczowe!
    class_weights=[1, 5],   # możesz dostroić
    random_seed=42,
    verbose=False
)

model.fit(X_train, y_train, eval_set=(X_test, y_test))

# ================================
# 8. Ewaluacja
# ================================
y_pred = model.predict(X_test)

print("\n=== Wyniki modelu ===\n")
print(classification_report(y_test, y_pred, digits=4))
print("\nMacierz pomyłek:")
print(confusion_matrix(y_test, y_pred))



: 